In [2]:
from amino_acids import aa, codons, aa_table
import random
import doctest

In [ ]:
aa[0]

In [ ]:
codons[10]

In [ ]:
aa_table

In [ ]:
myString = "asjh"
for character in myString:
    print(character)

In [ ]:
' '.join(random.sample(myString, len(myString)))

In [3]:
# A dictionary for returning the complements
completment_list = {'A':'T','C':'G', 'G':'C', 'T':'A'}
def get_complement(nucleotide):
    return completment_list[nucleotide.upper()]

In [ ]:
get_complement('C')

In [4]:
def get_reverse_complement(dna):
    """ Computes the reverse complementary sequence of DNA for the specfied DNA
        sequence

        dna: a DNA sequence represented as a string
        returns: the reverse complementary DNA sequence represented as a string
    >>> get_reverse_complement("ATGCCCGCTTT")
    'AAAGCGGGCAT'
    >>> get_reverse_complement("CCGCGTTCA")
    'TGAACGCGG'
    """
    # Create a empty string to hold the reverse complement
    reverse_complement = ''
    
    # Add the nucleotide complements to the reverse complement string
    for nucleotide in dna:
        reverse_complement += get_complement(nucleotide)
    return reverse_complement[::-1]

In [ ]:
get_reverse_complement("ATGCCCGCTTT")

In [ ]:
MYsTRING = "ATGCCCGCTTT"

In [ ]:
MYsTRING.find('ATG')
i = 0
while i < len(MYsTRING):
    old_i = i
    i += 3
    print(MYsTRING[old_i:i:1])

In [5]:
start_codon = codons[3][0]
stop_codon = codons[10]

def rest_of_ORF(dna):
    """ Takes a DNA sequence that is assumed to begin with a start
        codon and returns the sequence up to but not including the
        first in frame stop codon.  If there is no in frame stop codon,
        returns the whole string.

        dna: a DNA sequence
        returns: the open reading frame represented as a string
    >>> rest_of_ORF("ATGTGAA")
    'ATG'
    >>> rest_of_ORF("ATGAGATAGG")
    'ATGAGA'
    """
    i = 0
    ORF = ''
    
    # As long as the current position of the curser is not beyond the dna length,
    # read three as a group until hit a stop codon
    while i < len(dna):
        # Save the current i as the index to start recording 3 necleuctides
        old_i = i
        # Add 3 to i to get the end index
        i += 3
        # store the 3 necleuctides as one in a temporary variable
        temp = dna[old_i:i:1]
        
        # check if the codon is a stop codon, return the ORF as is; otherwise
        # keep adding codons to the ORF
        if temp == stop_codon[0] or temp == stop_codon[1] or temp == stop_codon[2]:
            return ORF
        else:
            ORF += temp
    return ORF

# import doctest
# doctest.testmod(verbose=True)

In [ ]:
rest_of_ORF("ATGCATGAATGTAGATAGATGTGCCC")

In [ ]:
newString = 'ATGCATGCGT'
newString[3:].find('ATG')

In [ ]:
myList = ['hello']
a = 'well'
myList.append(a)
myList

In [22]:
def find_all_ORFs_oneframe(dna):
    """ Finds all non-nested open reading frames in the given DNA
        sequence and returns them as a list.  This function should
        only find ORFs that are in the default frame of the sequence
        (i.e. they start on indices that are multiples of 3).
        By non-nested we mean that if an ORF occurs entirely within
        another ORF, it should not be included in the returned list of ORFs.

        dna: a DNA sequence
        returns: a list of non-nested ORFs
    >>> find_all_ORFs_oneframe("ATGCATGAATGTAGATAGATGTGCCC")
    ['ATGCATGAATGTAGA', 'ATGTGCCC']
    """
    ORF_list = []
    # base_frame is the base from which to look for a starting codon
    base_frame = dna
    # base_frame_num is the starting index for a start codon
    base_frame_num = base_frame.find(start_codon)
    
    # Making sure that the frames stay in sync by making sure the ATG location
    # can be divided by three
    while base_frame_num % 3 != 0:
        base_frame = base_frame[3:]
        base_frame_num = base_frame.find(start_codon)

        
    # Keep looking for an ORF until when there's no more to read or when it
    # would be a frame out of the original sync
    while 1:
        # base_frame_num would be negative when there is no starting codon to
        # be found, in this case, the loop should break and allow the function
        # to return the ORF collected up to this point
        if base_frame_num < 0:
            break
        # Get one full frame from rest_of_ORF() and append it to the ORF_list
        full_frame = rest_of_ORF(base_frame[base_frame_num:])
        ORF_list.append(full_frame)
        # Look for the next ATG index from a base_frame with the previously
        # found full_frame removed from the start
        next_ATG_loc = base_frame[len(full_frame):].find(start_codon)
        # if the index of the next ATG is not divisible by 3 (meaning that
        # the new start codon isn't in sync with the previuos one), then
        # break from the looking for another ORF
        if (next_ATG_loc % 3) != 0:
            break
        else:
            base_frame = base_frame[len(full_frame)+3:]
            base_frame_num = next_ATG_loc-3
    
    return ORF_list

doctest.testmod(verbose=True)

Trying:
    find_all_ORFs("ATGCATGAATGTAG")
Expecting:
    ['ATGCATGAATGTAG', 'ATGAATGTAG', 'ATG']
ok
Trying:
    find_all_ORFs_oneframe("ATGCATGAATGTAGATAGATGTGCCC")
Expecting:
    ['ATGCATGAATGTAGA', 'ATGTGCCC']
ok
Trying:
    get_reverse_complement("ATGCCCGCTTT")
Expecting:
    'AAAGCGGGCAT'
ok
Trying:
    get_reverse_complement("CCGCGTTCA")
Expecting:
    'TGAACGCGG'
ok
Trying:
    rest_of_ORF("ATGTGAA")
Expecting:
    'ATG'
ok
Trying:
    rest_of_ORF("ATGAGATAGG")
Expecting:
    'ATGAGA'
ok
2 items had no tests:
    __main__
    __main__.get_complement
4 items passed all tests:
   1 tests in __main__.find_all_ORFs
   1 tests in __main__.find_all_ORFs_oneframe
   2 tests in __main__.get_reverse_complement
   2 tests in __main__.rest_of_ORF
6 tests in 6 items.
6 passed and 0 failed.
Test passed.


TestResults(failed=0, attempted=6)

In [23]:
find_all_ORFs_oneframe("ATGCATGAATGTAGATAGATGTGCCC")

['ATGCATGAATGTAGA', 'ATGTGCCC']

In [24]:
len(find_all_ORFs_oneframe("ATGCATGAATGTAGATAGATGTGCCC"))
myString = "ATGCATGAATGTAGATAGATGTGCCC"
myString[2:]

'GCATGAATGTAGATAGATGTGCCC'

In [15]:
def find_all_ORFs(dna):
    """ Finds all non-nested open reading frames in the given DNA sequence in
        all 3 possible frames and returns them as a list.  By non-nested we
        mean that if an ORF occurs entirely within another ORF and they are
        both in the same frame, it should not be included in the returned list
        of ORFs.

        dna: a DNA sequence
        returns: a list of non-nested ORFs

    >>> find_all_ORFs("ATGCATGAATGTAG")
    ['ATGCATGAATGTAG', 'ATGAATGTAG', 'ATG']
    """
    all_ORF = []
    # Loop through the possibilities of a starting codon being on the multiples
    # of 0, 1, 2 index
    for i in range(3):
        temp_all_frame = find_all_ORFs_oneframe(dna[i:])
        # In case that there are more than just one frame in one find_all,
        # append individual ones to the list with the for loop
        for y in range(len(temp_all_frame)):
            all_ORF.append(temp_all_frame[y])
            
    return all_ORF

doctest.testmod(verbose=True)

Trying:
    find_all_ORFs("ATGCATGAATGTAG")
Expecting:
    ['ATGCATGAATGTAG', 'ATGAATGTAG', 'ATG']
ok
Trying:
    find_all_ORFs_oneframe("ATGCATGAATGTAGATAGATGTGCCC")
Expecting:
    ['ATGCATGAATGTAGA', 'ATGTGCCC']
ok
Trying:
    get_reverse_complement("ATGCCCGCTTT")
Expecting:
    'AAAGCGGGCAT'
ok
Trying:
    get_reverse_complement("CCGCGTTCA")
Expecting:
    'TGAACGCGG'
ok
Trying:
    rest_of_ORF("ATGTGAA")
Expecting:
    'ATG'
ok
Trying:
    rest_of_ORF("ATGAGATAGG")
Expecting:
    'ATGAGA'
ok
2 items had no tests:
    __main__
    __main__.get_complement
4 items passed all tests:
   1 tests in __main__.find_all_ORFs
   1 tests in __main__.find_all_ORFs_oneframe
   2 tests in __main__.get_reverse_complement
   2 tests in __main__.rest_of_ORF
6 tests in 6 items.
6 passed and 0 failed.
Test passed.


TestResults(failed=0, attempted=6)

In [ ]:
find_all_ORFs("ATGCATGAATGTAG")

In [ ]:
for i in range(3):
    print(i)

In [ ]:
a = 'ATGCATGAATGTAG'
find_all_ORFs_oneframe(a[2:])

In [ ]:
a[2:]

In [ ]:
rest_of_ORF('GCATGAATGTAG')

In [ ]:
b = 'ATGTAG'
b[6:].find('ATG')

In [ ]:
-1 % 3